# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather = pd.read_csv("output.csv")
weather.count()

City ID       550
City          550
Cloudiness    550
Country       547
Date          550
Humidity      550
Lat           550
Lng           550
Max_Temp      550
Wind_Speed    550
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = weather[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = weather["Humidity"].astype(float)
centerv= ["0","0"]
fig = gmaps.figure(zoom_level = 2, center = centerv)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# conditions = 
# temp lower than 80 but greater than 70
# speed less than 10
# clouds = 0
weather=weather.rename(columns={"Max_Temp":"Temp", "Wind_Speed":"Speed", })
# & 
weathersmall = weather.drop(weather[(weather['Cloudiness'] > 0)].index)
weathersmall = weathersmall.drop(weathersmall[(weathersmall['Speed'] > 10)].index)
weathersmall

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Speed
11,10,Yumen,0,CN,1579990335,69,40.28,97.20,13.50,4.59
23,21,Port Alfred,0,ZA,1579990272,98,-33.59,26.89,68.00,4.00
30,28,Plettenberg Bay,0,ZA,1579990611,91,-34.05,23.37,68.97,8.93
39,37,Qaanaaq,0,GL,1579990275,46,77.48,-69.36,-13.38,4.99
50,48,Nikolskoye,0,RU,1579990060,81,59.70,30.79,30.00,7.83
...,...,...,...,...,...,...,...,...,...,...
528,490,Dharampuri,0,IN,1579991435,45,22.17,75.35,61.27,6.11
537,496,Dunmore Town,0,BS,1579991450,67,25.50,-76.65,72.72,2.66
538,496,Bay Roberts,0,CA,1579991452,79,47.60,-53.26,19.00,4.14
540,498,Esna,0,EG,1579991455,78,25.29,32.55,42.58,2.84


In [6]:
hotel_df = weathersmall.loc[(weathersmall['Temp'] >= 70) & (weathersmall['Temp'] <80)]
hotel_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Speed
129,118,East London,0,ZA,1579990288,85,-33.02,27.91,77.31,0.51
148,137,Castro,0,BR,1579990301,53,-24.79,-50.01,77.29,0.98
167,154,Waipawa,0,NZ,1579990837,79,-41.41,175.52,71.01,5.01
194,179,Valparaiso,0,CL,1579990659,51,-33.04,-71.63,71.53,8.61
229,211,Belle Glade,0,US,1579990942,53,26.68,-80.67,75.99,4.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df["Hotel Name"]=""
hotel_df.head()

C:\Users\steel\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Speed,Hotel Name
129,118,East London,0,ZA,1579990288,85,-33.02,27.91,77.31,0.51,
148,137,Castro,0,BR,1579990301,53,-24.79,-50.01,77.29,0.98,
167,154,Waipawa,0,NZ,1579990837,79,-41.41,175.52,71.01,5.01,
194,179,Valparaiso,0,CL,1579990659,51,-33.04,-71.63,71.53,8.61,
229,211,Belle Glade,0,US,1579990942,53,26.68,-80.67,75.99,4.00,


In [9]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\Users\steel\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [10]:
hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Speed,Hotel Name
129,118,East London,0,ZA,1579990288,85,-33.02,27.91,77.31,0.51,Premier Hotels & Resorts
148,137,Castro,0,BR,1579990301,53,-24.79,-50.01,77.29,0.98,CHACARA BAILLY
167,154,Waipawa,0,NZ,1579990837,79,-41.41,175.52,71.01,5.01,
194,179,Valparaiso,0,CL,1579990659,51,-33.04,-71.63,71.53,8.61,Hotel Palacio Astoreca
229,211,Belle Glade,0,US,1579990942,53,26.68,-80.67,75.99,4.00,Horizon Inn & Suites
242,224,Road Town,0,VG,1579990964,84,18.42,-64.62,77.00,4.00,Wyndham Tortola BVI Lambert Beach Resort
245,227,Sungai Udang,0,MY,1579990969,83,2.27,102.14,77.00,5.01,Hotel Pantai Puteri
280,259,Bonthe,0,SL,1579990957,86,7.53,-12.51,78.10,8.41,Bonthe Holiday Village
368,344,Cockburn Town,0,TC,1579990953,65,21.46,-71.14,76.35,6.38,Bohio Dive Resort
382,356,Goure,0,NE,1579991194,14,13.98,10.27,70.00,10.00,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker_locations = locations
markers = gmaps.marker_layer(marker_locations, hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))